In [26]:
#import library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [27]:
#read data
df = pd.read_csv("train.csv")
df.head()

,ID,Age,Gender,Education_Status,Employment_Status,Working_Week (Yearly),Industry_Status,Occupation_Status,Race,Hispanic_Origin,...,Citizenship,Birth_Country,Birth_Country (Father),Birth_Country (Mother),Tax_Status,Gains,Losses,Dividends,Income_Status,Income
0,TRAIN_00000,63,M,Middle (7-8),Full-Time,4,Social Services,Services,White,All other,...,Native,US,US,US,Nonfiler,0,0,0,Unknown,425
1,TRAIN_00001,37,M,Associates degree (Vocational),Full-Time,52,Entertainment,Services,White,All other,...,Native,US,US,US,Single,0,0,0,Under Median,0
2,TRAIN_00002,58,F,High graduate,Full-Time,52,Manufacturing (Non-durable),Admin Support (include Clerical),Black,All other,...,Native,US,US,US,Married Filling Jointly both under 65 (MFJ),3411,0,0,Under Median,860
3,TRAIN_00003,44,M,High graduate,Full-Time,52,Retail,Technicians & Support,White,All other,...,Native,US,US,US,Single,0,0,0,Under Median,850
4,TRAIN_00004,37,F,High graduate,Full-Time,52,Retail,Sales,White,All other,...,Native,US,US,US,Head of Household (HOH),0,0,0,Unknown,570


In [28]:

# 국가를 대륙으로 매핑하는 딕셔너리
country_continent_dict = {
 'US': 'US',
 'Cuba': 'North America',
 'Portugal': 'Europe',
 'Mexico': 'North America',
 'Unknown': 'US',
 'Puerto-Rico': 'North America',
 'Germany': 'Europe',
 'Japan': 'Asia',
 'Poland': 'Europe',
 'Columbia': 'South America',
 'Philippines': 'Asia',
 'Italy': 'Europe',
 'Trinadad&Tobago': 'South America',
 'England': 'Europe',
 'South Korea': 'Asia',
 'Iran': 'Asia',
 'France': 'Europe',
 'India': 'Asia',
 'China': 'Asia',
 'Dominican-Republic': 'North America',
 'Scotland': 'Europe',
 'Ecuador': 'South America',
 'Nicaragua': 'North America',
 'Peru': 'South America',
 'Cambodia': 'Asia',
 'Canada': 'North America',
 'Jamaica': 'North America',
 'Vietnam': 'Asia',
 'Hong Kong': 'Asia',
 'Thailand': 'Asia',
 'Haiti': 'North America',
 'Guatemala': 'North America',
 'Laos': 'Asia',
 'Yugoslavia': 'Europe',
 'Ireland': 'Europe',
 'El-Salvador': 'North America',
 'Panama': 'North America',
 'Honduras': 'North America',
 'Greece': 'Europe',
 'Outlying-U S (Guam USVI etc)': 'US',
 'Hungary': 'Europe',
 'Taiwan': 'Asia',
 'Holand-Netherlands': 'Europe'
}

# 업데이트할 열 목록: 본인 출신국가, 엄마 출신국가, 아빠 출신국가
columns_to_update = ['Birth_Country', 'Birth_Country (Mother)', 'Birth_Country (Father)']

# 각 열에 대해 국가를 대륙으로 매핑
for column in columns_to_update:
    df[column] = df[column].map(country_continent_dict)


            ID  Age Gender                Education_Status Employment_Status  \
0  TRAIN_00000   63      M                    Middle (7-8)         Full-Time   
1  TRAIN_00001   37      M  Associates degree (Vocational)         Full-Time   
2  TRAIN_00002   58      F                   High graduate         Full-Time   
3  TRAIN_00003   44      M                   High graduate         Full-Time   
4  TRAIN_00004   37      F                   High graduate         Full-Time   

   Working_Week (Yearly)              Industry_Status  \
0                      4              Social Services   
1                     52                Entertainment   
2                     52  Manufacturing (Non-durable)   
3                     52                       Retail   
4                     52                       Retail   

                  Occupation_Status   Race Hispanic_Origin  ... Citizenship  \
0                          Services  White       All other  ...      Native   
1                 

In [29]:
# Remove rows where 'Age' is below 17 or above 75
df = df[df['Age'].between(17, 75)]

# Remove rows with 'Employment Status' as 'Not Working' or 'Seeking Full-Time'
df = df[~df['Employment_Status'].isin(['Not Working', 'Seeking Full-Time'])]

# Drop the 'Gains', 'Losses', 'Dividends', 'Household_Status', 'Income_Status' columns
df.drop(['Gains', 'Losses', 'Dividends', 'Household_Status', 'Income_Status'], axis=1, inplace=True)

# Map 'Gender' values from 'M' and 'F' to 0 and 1, respectively
df['Gender'] = df['Gender'].map({'M': 0, 'F': 1})

# Consolidate education levels and rename as specified
education_map = {
    'High graduate': 'High', 'High Senior': 'High', 
    'High Junior': 'High', 'High Sophomore': 'High',
    'Elementary (5-6)': 'Elementary(1-6)', 'Elementary (1-4)': 'Elementary(1-6)',
    'Kindergarten': 'Baby', 'Children': 'Baby'
}
df['Education_Status'] = df['Education_Status'].replace(education_map)


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15851 entries, 0 to 19999
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   ID                      15851 non-null  object
 1   Age                     15851 non-null  int64 
 2   Gender                  15851 non-null  int64 
 3   Education_Status        15851 non-null  object
 4   Employment_Status       15851 non-null  object
 5   Working_Week (Yearly)   15851 non-null  int64 
 6   Industry_Status         15851 non-null  object
 7   Occupation_Status       15851 non-null  object
 8   Race                    15851 non-null  object
 9   Hispanic_Origin         15851 non-null  object
 10  Martial_Status          15851 non-null  object
 11  Household_Summary       15851 non-null  object
 12  Citizenship             15851 non-null  object
 13  Birth_Country           15851 non-null  object
 14  Birth_Country (Father)  15851 non-null  object
 15  Birth_C

In [33]:
columns=[
    'Education_Status',
    'Employment_Status',
    'Industry_Status',
    'Occupation_Status',
    'Race',
    'Hispanic_Origin',
    'Martial_Status',
    'Household_Summary',
    'Citizenship',
    'Birth_Country',
    'Birth_Country (Father)',
    'Birth_Country (Mother)',
    'Tax_Status'
]

In [34]:
df_encoded=pd.get_dummies(df,columns=columns,dtype=int)

In [35]:
#제거코드(ID)
df_encoded = df_encoded.drop(columns=['ID'])

In [36]:
df_encoded.to_csv("final.csv", index=False)

In [37]:
# 필요한 라이브러리 임포트
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# 데이터 불러오기


# 데이터 전처리 (가상의 과정)
# 예시: 'Age'와 'Income'을 제외한 모든 열을 범주형 변수로 간주하고 원-핫 인코딩 수행

ready_X = df_encoded
ready_y = df['Income']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(ready_X, ready_y, test_size=0.2, random_state=42)

# 모델 초기화 및 학습
rf = RandomForestRegressor(n_estimators=100, random_state=50)
rf.fit(X_train, y_train)


RandomForestRegressor(random_state=50)

In [38]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, GridSearchCV

# 모델 학습
rf.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = rf.predict(X_test)

# 성능 평가
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"R²: {r2}")

# # 하이퍼파라미터 튜닝 (예시)
# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'max_depth': [None, 10, 20, 30],
# }
# grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
# grid_search.fit(X_train, y_train)

# # 최적의 파라미터와 그 때의 성능
# print(f"Best parameters: {grid_search.best_params_}")
# print(f"Best score (RMSE): {(-grid_search.best_score_) ** 0.5}")


RMSE: 8.718831006966209
R²: 0.9998612546859453


## 여기서부터 테스트 데이터를 가공해서 모델에 집어 넣었음


---



In [40]:

test_df = pd.read_csv("Editedtest.csv")
test_df.head()

,Age,Gender,Working_Week (Yearly),Education_Status_Associates degree (Academic),Education_Status_Associates degree (Vocational),Education_Status_Baby,Education_Status_Bachelors degree,Education_Status_College,Education_Status_Doctorate degree,Education_Status_Elementary(1-6),...,Birth_Country (Mother)_Europe,Birth_Country (Mother)_North America,Birth_Country (Mother)_South America,Birth_Country (Mother)_US,Tax_Status_Head of Household (HOH),Tax_Status_Married Filling Jointly both over 65 (MFJ),Tax_Status_Married Filling Jointly both under 65 (MFJ),Tax_Status_Married Filling Jointly one over 65 & one under 65 (MFJ),Tax_Status_Nonfiler,Tax_Status_Single
0,47,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,1,0
1,18,1,52,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,39,1,30,0,1,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
3,31,0,24,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
4,17,0,51,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [49]:
y_pred = rf.predict(test_df)

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- Household_Summary_Child under 18 ever married
- Income


In [45]:
## 테스트
# column_names = df_encoded.columns.tolist()
# print(column_names)

['Age', 'Gender', 'Working_Week (Yearly)', 'Income', 'Education_Status_Associates degree (Academic)', 'Education_Status_Associates degree (Vocational)', 'Education_Status_Baby', 'Education_Status_Bachelors degree', 'Education_Status_College', 'Education_Status_Doctorate degree', 'Education_Status_Elementary(1-6)', 'Education_Status_High', 'Education_Status_High Freshman', 'Education_Status_Masters degree', 'Education_Status_Middle (7-8)', 'Education_Status_Professional degree', 'Employment_Status_Children or Armed Forces', 'Employment_Status_Choice Part-Time', 'Employment_Status_Full-Time', 'Employment_Status_Part-Time (Usually Full-Time)', 'Employment_Status_Part-Time (Usually Part-Time)', 'Employment_Status_Seeking Part-Time', 'Industry_Status_Agriculture', 'Industry_Status_Business & Repair', 'Industry_Status_Communications', 'Industry_Status_Construction', 'Industry_Status_Education', 'Industry_Status_Entertainment', 'Industry_Status_Finance Insurance & Real Estate', 'Industry_Stat

In [46]:
# # column_names에 해당하는 열만 남기고 나머지는 삭제
# columns_to_encode = [col for col in df.columns if col in column_names]
# df_to_encode = df[columns_to_encode]

# # 원핫인코딩 수행
# encoded_df = pd.get_dummies(df_to_encode)